In [34]:
#version2
from ultralytics import YOLO
from PIL import Image
import os
import cv2
import numpy as np
img_dir = "C:/Users/jaeho/yolo_project/sample"
save_dir = 'C:/Users/jaeho/yolo_project/bestpt'
result_dir = 'C:/Users/jaeho/yolo_project/result'


model1 = YOLO("polygon_200_best.pt")
model2 = YOLO("epoch_100_bbox_best.pt")

for im_path in os.listdir(img_dir):
    image_list = cv2.imread(os.path.join(img_dir, im_path))
    predict1 = model1.predict(image_list, conf=0.2)
    predict2 = model2.predict(image_list, conf=0.2)
    print(predict1)
    result1 = predict1[0]
    result2 = predict2[0]


    result1.boxes = []
    result_image = Image.fromarray(result1.plot()[:,:,::-1])
    path = os.path.join(save_dir, f"{im_path}")
    result_image.save(path)
    image = cv2.imread(os.path.join(save_dir, im_path))
    bbox = result2.boxes.xyxy.cpu().numpy()
    im_name = im_path.replace(".jpg", "")

    for coord in bbox:
        x1, y1, x2, y2 = map(int, coord)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 초록색 사각형 그리기
    
    image_to_save = np.array(image)
    output_image_path = f'C:/Users/jaeho/yolo_project/result/{im_name}_D.jpg'
    cv2.imwrite(output_image_path, image_to_save)
    


0: 640x640 2 sideWalks, 1 crossWalk, 1 bikeLane, 1 intersection, 1 stopLine, 1 violationStopLine, 621.4ms
Speed: 3.5ms preprocess, 621.4ms inference, 8.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 helmet, 5 redLights, 1 ridingMotor, 500.3ms
Speed: 2.0ms preprocess, 500.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
keys: ['boxes', 'masks']
masks: ultralytics.engine.results.Masks object
names: {0: 'sideWalk', 1: 'crossWalk', 2: 'bikeLane', 3: 'intersection', 4: 'stopLine', 5: 'violationStopLine'}
orig_img: array([[[123, 129, 112],
        [ 89,  95,  78],
        [ 79,  82,  66],
        ...,
        [ 63,  51,  49],
        [ 65,  53,  51],
        [ 67,  55,  53]],

       [[122, 128, 111],
        [ 92,  98,  81],
        [ 80,  83,  67],
        ...,
        [ 66,  54,  52],
        [ 69,  57,  55],
        [ 71,  59,  57]],

       [[109, 112,  96],
        [ 87,  90,  74],
        [ 75,  78,  62],
        ...,
        [ 62,  52,  52],
        [ 64,  54,  54],
        [ 66,  56,  56]],

       ...,

       [[ 55,  48,  51],
        [ 62,  55,  58],
        [ 67,  59,  60],
        ...,
        [ 60,  78,  77],
        [ 73,  94,  92],
        [ 72,  94,  92]],

       [[

In [78]:
# 인도 주행 완료!
save_dir = "C:/Users/jaeho/yolo_project/result"
image_dir = "C:/Users/jaeho/yolo_project/sample"
count = 1
# # List all files in the directory
image_files = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]
import cv2
import numpy as np

def is_bottom_in_mask(box, masks_xy):
    x_min, y_min, x_max, y_max = box
    bottom_left = (x_min, y_max)
    bottom_right = (x_max, y_max)

    for mask_xy in masks_xy:
        mask_xy = mask_xy
        mask_xy = mask_xy.reshape(-1, 1, 2)
        if cv2.pointPolygonTest(mask_xy, bottom_left, False) >= 0 and cv2.pointPolygonTest(mask_xy, bottom_right, False) >= 0:
            return True
    return False

for image_path in image_files:
    image = cv2.imread(image_path)
    results1 = model2.predict(image_path, save=True, conf=0.12)
    results2 = model1.predict(image_path, save=True, conf=0.12)
    result_image = Image.fromarray(result1.plot()[:,:,::-1])
    
    count = count + 1
    path = os.path.join(save_dir, f"{count}_P.jpg")
    result_image.save(path)

    for result1, result2 in zip(results1, results2):
        boxes_xyxy = result1.boxes.xyxy.numpy()
        labels = result1.boxes.cls.numpy()
        masks_xy = result2.masks.xy
        masks_cls = result2.boxes.cls.numpy()
        
        # 보도 클래스에 대한 마스크만 가져옵니다.
        sidewalk_masks_xy = [masks_xy[i] for i in range(len(masks_cls)) if masks_cls[i] == 0]
        
        # 차량 클래스에 대한 박스만 가져옵니다.
        vehicle_boxes_xyxy = boxes_xyxy[np.isin(labels, [8, 9, 10])]
        
        for box in vehicle_boxes_xyxy:
            if is_bottom_in_mask(box, sidewalk_masks_xy):
                print(f"The bottom of the box in {image_path} is in a mask!")
                print(f"Coordinates of the box: {box}")
                print(image)
                x_min, y_min, x_max, y_max = map(int, box)
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
                
        # Save the image
        cv2.imwrite(f'C:/Users/jaeho/yolo_project/result2/{os.path.basename(image_path)}', image)


    

image 1/1 C:\Users\jaeho\yolo_project\sample\C000002_055_0066_C_D_F_0_jpg.rf.c4145fcf059658719581bc0e9c5a2326.jpg: 640x640 1 greenLight, 1 ridingMotor, 503.8ms
Speed: 2.0ms preprocess, 503.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict4

image 1/1 C:\Users\jaeho\yolo_project\sample\C000002_055_0066_C_D_F_0_jpg.rf.c4145fcf059658719581bc0e9c5a2326.jpg: 640x640 2 sideWalks, 1 crossWalk, 1 intersection, 1 stopLine, 1 violationStopLine, 653.6ms
Speed: 2.5ms preprocess, 653.6ms inference, 8.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\predict3

image 1/1 C:\Users\jaeho\yolo_project\sample\C000002_058_0144_C_D_F_0_jpg.rf.b30ea84d0fe76de11fb374d275083947.jpg: 640x640 2 helmets, 2 ridingMotors, 521.9ms
Speed: 3.0ms preprocess, 521.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict4

image 1/1 C:\Users\jaeho\yolo_project\sample\C000002_058_0144_C_

이동수단 + 사람 -> 헬멧 유무
1. 이동수단좌표와 사람좌표가 겹친 상태와 헬멧을 좌표를 이용하여 헬멧 유무 판단
 반복문으로 먼저 



In [84]:
# 오토바이 헬멧 
image_dir = "C:/Users/jaeho/yolo_project/sample"
save_dir = "C:/Users/jaeho/yolo_project/result"
count = 1
# # List all files in the directory
image_files = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]

import numpy as np

def is_overlapping(box1, box2):
    # box = (x1, y1, x2, y2)
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # Check if one box is to the right or left of the other
    if x1_1 > x2_2 or x1_2 > x2_1:
        return False

    # Check if one box is above or below the other
    if y1_1 > y2_2 or y1_2 > y2_1:
        return False

    return True

def get_center(box):
    # box = (x1, y1, x2, y2)
    return (box[0]+box[2])/2, (box[1]+box[3])/2

def get_distance(box1, box2):
    center1 = get_center(box1)
    center2 = get_center(box2)
    return np.sqrt((center1[0]-center2[0])**2 + (center1[1]-center2[1])**2)


import cv2

# ...

for image_path in image_files:
    image = cv2.imread(image_path)
    results = model2.predict(image_path, save=True, conf=0.12)
    result_image = Image.fromarray(results.plot()[:,:,::-1])
    count = count + 1
    path = os.path.join(save_dir, f"{count}_P.jpg")
    result_image.save(path)
    
    
    for result in results:
        boxes_xyxy = result.boxes.xyxy.numpy()
        labels = result.boxes.cls.numpy()
        
        riding_motorcycles, helmets = [], []

        for i, box in enumerate(boxes_xyxy):
            label = labels[i]
            if label == 10:
                riding_motorcycles.append(box)
            elif label == 3:
                helmets.append(box)

        for helmet in helmets:
            # find the closest motorcycle rider to the helmet
            distances = [get_distance(helmet, riding_motorcycle) for riding_motorcycle in riding_motorcycles]
            closest_riding_motorcycle = riding_motorcycles[np.argmin(distances)]

            # check if this motorcycle rider is wearing the helmet
            if is_overlapping(closest_riding_motorcycle, helmet):
                print("Okay, pass")
            else:
                print("A motorcycle rider is violating the rules!")
                x1, y1, x2, y2 = closest_riding_motorcycle.astype(int)
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    
    # save the image
    result_image_path = os.path.join("C:/Users/jaeho/yolo_project/result2", os.path.basename(image_path))
    cv2.imwrite(result_image_path, image)


image 1/1 C:\Users\jaeho\yolo_project\sample\C000002_055_0066_C_D_F_0_jpg.rf.c4145fcf059658719581bc0e9c5a2326.jpg: 640x640 1 greenLight, 1 ridingMotor, 506.4ms
Speed: 3.0ms preprocess, 506.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict4


AttributeError: 'list' object has no attribute 'plot'

In [ ]:
from ultralytics import YOLO
from PIL import Image
import os
import cv2
import numpy as np

# Initialize the models (adjust paths as necessary)
model1 = YOLO('path/to/your/first/model/file')
model2 = YOLO('path/to/your/second/model/file')

save_dir = "/Users/parkyunsu/gitfile/cvp/algorithm/result"
image_dir = "/Users/parkyunsu/gitfile/cvp/algorithm/sample"
count = 1

# List all files in the directory
image_files = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]

import cv2
import numpy as np

def is_bottom_in_mask(box, masks_xy):
    x_min, y_min, x_max, y_max = box
    bottom_left = (x_min, y_max)
    bottom_right = (x_max, y_max)

    for mask_xy in masks_xy:
        mask_xy = mask_xy.reshape(-1, 1, 2)
        if cv2.pointPolygonTest(mask_xy, bottom_left, False) >= 0 and cv2.pointPolygonTest(mask_xy, bottom_right, False) >= 0:
            return True
    return False

for image_path in image_files:
    image = cv2.imread(image_path)
    results1 = model2.predict(image_path, save=True, conf=0.12)
    results2 = model1.predict(image_path, save=True, conf=0.12)
    
    # Ensure both result lists have the same length
    if len(results1) != len(results2):
        print(f"Warning: results1 and results2 have different lengths for image {image_path}")
        continue

    for result1, result2 in zip(results1, results2):
        boxes_xyxy = result1.boxes.xyxy.numpy()
        labels = result1.boxes.cls.numpy()
        
        if len(result2) == 0:
            print(f"{image_path} 폴리곤 객체가 없습니다. 스킵합니다!")
            continue
        
        masks_xy = result2.masks.xy
        masks_cls = result2.boxes.cls.numpy()
        
        # 횡단보도 클래스에 대한 마스크만 가져옵니다.
        sidewalk_masks_xy = [masks_xy[i] for i in range(len(masks_cls)) if masks_cls[i] == 1]
        
        # 탈거 객체 클래스에 대한 박스만 가져옵니다.
        vehicle_boxes_xyxy = boxes_xyxy[np.isin(labels, [8, 9, 10])]
        
        # 보행자 신호 클래스에 대한 박스만 가져옵니다.
        pedlight_boxes_xyxy = boxes_xyxy[np.isin(labels, [5, 6])] # 5: pedGreen, 6: pedRed
        
        for box in vehicle_boxes_xyxy:
            if is_bottom_in_mask(box, sidewalk_masks_xy):
                if 6 in masks_cls:  # Check if pedRed is present
                    print("횡단보도 위 객체 -> 무단횡단")
                    box_color = (0, 0, 255)  # 빨간색
                elif 5 in masks_cls:  # Check if pedGreen is present
                    print("횡단보도 위 객체 -> 횡단보도 주행")
                    box_color = (0, 255, 0)  # 초록색
                else:
                    print("신호등이 안보입니다. 위반 사항을 확인하십시오.")
                    box_color = (255, 0, 0)  # 파란색

                print(f"The bottom of the box in {image_path} is in a mask!")
                print(f"Coordinates of the box: {box}")
                print(image)
                x_min, y_min, x_max, y_max = map(int, box)
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), box_color, 2)
                
        # Save the image
        cv2.imwrite(f'/Users/parkyunsu/gitfile/cvp/algorithm/신호등주행/{os.path.basename(image_path)}', image)


In [ ]:
import os
import cv2
from PIL import Image
import numpy as np

save_dir = "C:/Users/jaeho/yolo_project/result"
image_dir = "C:/Users/jaeho/yolo_project/sample"
count = 1

# List all files in the directory
image_files = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]

def is_bottom_in_mask(box, masks_xy):
    x_min, y_min, x_max, y_max = box
    bottom_left = (x_min, y_max)
    bottom_right = (x_max, y_max)

    for mask_xy in masks_xy:
        mask_xy = mask_xy.reshape(-1, 1, 2)
        if cv2.pointPolygonTest(mask_xy, bottom_left, False) >= 0 and cv2.pointPolygonTest(mask_xy, bottom_right, False) >= 0:
            return True
    return False

def check_violation(box, red_light_mask, crosswalk_mask, stopline_mask, green_light_detected):
    if red_light_mask is not None and (is_bottom_in_mask(box, [red_light_mask])):
        if crosswalk_mask is not None and is_bottom_in_mask(box, [crosswalk_mask]):
            return "Crosswalk Violation"
        elif stopline_mask is not None and is_bottom_in_mask(box, [stopline_mask]):
            return "Stopline Violation"
    elif green_light_detected:
        return "Okay Pass"
    return None

for image_path in image_files:
    image = cv2.imread(image_path)
    results1 = model2.predict(image_path, save=True, conf=0.12)  # bbox
    results2 = model1.predict(image_path, save=True, conf=0.12)  # poly

    result_image = Image.fromarray(results1.plot()[:,:,::-1])
    
    count = count + 1
    path = os.path.join(save_dir, f"{count}_P.jpg")
    result_image.save(path)

    if len(results2) == 0:
        print(f"{image_path} has no polygon objects. Skipping!")
        continue

    masks_xy = results2.masks.xy
    masks_cls = results2.boxes.cls.numpy()
    
    red_light_mask = masks_xy[np.where(masks_cls == 7)[0][0]] if 7 in masks_cls else None
    crosswalk_mask = masks_xy[np.where(masks_cls == 1)[0][0]] if 1 in masks_cls else None
    stopline_mask = masks_xy[np.where(masks_cls == 5)[0][0]] if 5 in masks_cls else None
    green_light_detected = 2 in masks_cls

    for result1 in results1:
        boxes_xyxy = result1.boxes.xyxy.numpy()
        labels = result1.boxes.cls.numpy()

        vehicle_boxes_xyxy = boxes_xyxy[np.isin(labels, [8, 9, 10])]
        
        for box in vehicle_boxes_xyxy:
            violation = check_violation(box, red_light_mask, crosswalk_mask, stopline_mask, green_light_detected)
            if violation:
                print(f"Violation detected in {image_path}: {violation}")
                x_min, y_min, x_max, y_max = map(int, box)
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)

        cv2.imwrite(f'C:/Users/jaeho/yolo_project/sample/{os.path.basename(image_path)}', image)